In [1]:
# @title <font color="\#8FBC8F">Imports
# 
import os
import numpy as np


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100

!pip install mne --upgrade --quiet 
import mne

from mne.time_frequency import tfr_morlet as morl

print('[imports successfully loaded]')

     |████████████████████████████████| 7.0MB 5.4MB/s 
[imports successfully loaded]


In [3]:
#@title <font color="\#8FBC8F">File loading utility code
# choose subject index to analize
subject_index = 9#@param {type:'integer'} 
DATA_DIR = r'/content/drive/MyDrive/Colab Notebooks/Project Domino/new Macros/'
subject_list = sorted([f for f in os.listdir(DATA_DIR) if 'sub' in f])
subject_path = DATA_DIR + f'{subject_list[subject_index]}/'
subject_files = sorted(os.listdir(subject_path))
print(f'[Working on {subject_list[subject_index]}]')

print(f'subject\'s files: {subject_files}')

[Working on sub-030]
subject's files: ['sub-030_Baseline_clean_epo.fif', 'sub-030_Trials_clean_epo.fif', 'sub-030_channel_names.npy', 'sub-030_label_0_match_mean.npy', 'sub-030_label_1_non-match_mean.npy', 'sub-030_y.npy']


In [4]:
#@title <font color="\#8FBC8F">Load baseline and trials Epochs

verbose = False #@param {type:'boolean'}

bl = mne.read_epochs(subject_path + subject_files[0], verbose=verbose)
tr = mne.read_epochs(subject_path + subject_files[1], verbose=verbose)

# Picked_match=0, Picked_non-match=1
y = tr.events[:,2] - 13

print(f'[Baseline and Trials Loaded successfully]')

[Baseline and Trials Loaded successfully]


In [ ]:
# tr.events

In [5]:
#@title <font color="\#8FBC8F">Calculate Wavelet Transform for baselines and trials data (takes time)

n_freqs = 50
freqs = np.geomspace(1, 80, n_freqs)
n_cycles = np.geomspace(1, 12, n_freqs)

baseline = morl(bl, freqs=freqs, n_cycles=n_cycles, average=False, return_itc=False)
trials =   morl(tr, freqs=freqs, n_cycles=n_cycles, average=False, return_itc=False)

print(f'\n[successfull, avoid recalculating if possible]')

Not setting metadata
Not setting metadata

[successfull, avoid recalculating if possible]


In [ ]:
# baseline.data.shape,trials.data.shape

In [ ]:
# len(tr.ch_names)

In [6]:
#@title <font color="\#8FBC8F">Calculate X <font size=3, color=white>(Trials Epochs normalized by Baselines)

bl_data, tr_data = baseline.data[:1001], trials.data
b = np.repeat(np.median(bl_data, axis=3)[:, :, :, np.newaxis], 
              tr_data.shape[3], axis=3)

del bl_data, baseline

X = tr_data / b

del tr_data, trials
print(f'[X calculated successfully]')

[X calculated successfully]


In [6]:
#@title <font color="\#8FBC8F">Plot WT function definition

def plot_one_epoch_channel(ep, epochs, ep_num=0, ch_num=0):
    times = ([round(n, ndigits=4) for n in np.arange(ep.tmin, ep.tmax+1/250, 1/250)])
    n_points = 20; step = 1000//n_points
    data = epochs
    if not isinstance(data, np.ndarray): data = epochs.data[:,:,:,:1001]
    _ = plt.figure(figsize=(8,8))
    _ = plt.imshow(data[ep_num, ch_num], aspect='auto')
    _ = plt.xticks(ticks=range(0, data.shape[3], step), labels=times[::step], rotation=90)
    _ = plt.yticks(ticks=range(n_freqs)[::2], labels=np.array([round(n, ndigits=4) for n in freqs])[::2])
    _ = plt.colorbar()
    _ = plt.xlabel('Time [s]')
    _ = plt.ylabel('Frequency [Hz]')
print(f'[Function defined]')

[Function defined]


In [7]:
#@title <font color="\#8FBC8F">Plot WT <font size=3, color=white>(Baseline, Baseline mean, Trials, Normalized Trials)
plot = False #@param {type:'boolean'}
epoch_number =  0#@param {type:'integer'}
channel_number = 0 #@param {type:'integer'}

def plot_stuff():
    plot_one_epoch_channel(bl, baseline, epoch_number, channel_number)
    _ = plt.title('Baseline wavelet transform')

    plot_one_epoch_channel(tr, b, epoch_number, channel_number)
    _ = plt.title('Normalization mask')

    plot_one_epoch_channel(tr, trials, epoch_number, channel_number)
    _ = plt.title('Trials wavelet')

    plot_one_epoch_channel(tr, X, epoch_number, channel_number)
    _ = plt.title('Normalized wavelet output')

if plot: plot_stuff()

In [7]:
#@title calculate WT ERP-Like matrices
X_0 = X[np.where(y==0)].mean(axis=0)[:,::-1,:]
X_1 = X[np.where(y==1)].mean(axis=0)[:,::-1,:]

In [8]:
#@title <font color="\#8FBC8F">Save X&y

names = [subject_path + f'{subject_list[subject_index]}_X.npy',
         subject_path + f'{subject_list[subject_index]}_y.npy',
         subject_path + f'{subject_list[subject_index]}_channel_names.npy',
         subject_path + f'{subject_list[subject_index]}_label_0_match_mean.npy',
         subject_path + f'{subject_list[subject_index]}_label_1_non-match_mean.npy']

arrays = [X,y,tr.ch_names, X_0, X_1]

print(f'[Saving files for {subject_list[subject_index]}]\n')
for f,a in zip(names,arrays):print(f'[{f}]'); np.save(f,a)
print('\n[X, y and channel_names saved successfully]')



[Saving files for sub-030]

[/content/drive/MyDrive/Colab Notebooks/Project Domino/new Macros/sub-030/sub-030_X.npy]
[/content/drive/MyDrive/Colab Notebooks/Project Domino/new Macros/sub-030/sub-030_y.npy]
[/content/drive/MyDrive/Colab Notebooks/Project Domino/new Macros/sub-030/sub-030_channel_names.npy]
[/content/drive/MyDrive/Colab Notebooks/Project Domino/new Macros/sub-030/sub-030_label_0_match_mean.npy]
[/content/drive/MyDrive/Colab Notebooks/Project Domino/new Macros/sub-030/sub-030_label_1_non-match_mean.npy]

[X, y and channel_names saved successfully]


In [ ]:
clear_memory = False #@param {type:'boolean'}

if clear_memory:
    del bl, tr, X, b
    print('[Memory cleared]')

In [ ]:
# plt.figure()
# plt.imshow(X_0[0], aspect='auto')

# plt.figure()
# _ = plt.imshow(X_1[0], aspect='auto')
